# TikTok Ad Transparency Data Analysis

This notebook extracts and analyzes data from the TikTok Ads API, including:
- Ad campaign details
- Creative content (images and videos)
- Performance metrics
- Targeting information

In [ ]:
# Install required packages
!pip install pandas tqdm requests pillow xlsxwriter python-dateutil

In [ ]:
# Import libraries and setup
import pandas as pd
import os
from datetime import datetime, timedelta
import requests
from PIL import Image
import concurrent.futures
from tqdm import tqdm
import json
from dateutil.parser import parse
import warnings
warnings.filterwarnings('ignore')

# TikTok API Configuration
class TikTokAdConfig:
    def __init__(self):
        self.API_KEY = None  # Will be set by user
        self.ADVERTISER_ID = None  # Will be set by user
        self.BASE_URL = "https://business-api.tiktok.com/open_api/v1.3/ad/report/"
        self.HEADERS = None  # Will be set after API key is provided
    
    def setup(self, api_key, advertiser_id):
        """Setup TikTok API configuration"""
        self.API_KEY = api_key
        self.ADVERTISER_ID = advertiser_id
        self.HEADERS = {
            "Access-Token": self.API_KEY,
            "Content-Type": "application/json"
        }
        return True

# Create base output directory
BASE_OUTPUT_DIR = os.path.join(os.path.expanduser('~'), 'CascadeProjects', 'tiktok_ad_transparency')
output_dir = os.path.join(BASE_OUTPUT_DIR, f"tiktok_ads_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
os.makedirs(output_dir, exist_ok=True)

# Initialize TikTok configuration
tiktok_config = TikTokAdConfig()

print("✅ Setup complete!")
print(f"📁 Output directory: {output_dir}")

In [ ]:
# Define fields to retrieve
def get_main_fields():
    """Define the main fields to retrieve from TikTok Ads API"""
    return [
        # Ad Identification
        "ad_id",
        "ad_name",
        "campaign_id",
        "campaign_name",
        "adgroup_id",
        "adgroup_name",
        
        # Creative Information
        "image_ids",
        "video_id",
        "creative_type",
        "creative_format",
        "display_url",
        "landing_page_url",
        
        # Advertiser Information
        "advertiser_id",
        "advertiser_name",
        "industry",
        "secondary_industry",
        
        # Targeting Information
        "age_groups",
        "gender",
        "languages",
        "locations",
        "interests",
        "behaviors",
        
        # Performance Metrics
        "impressions",
        "clicks",
        "conversions",
        "spend",
        "ctr",
        "cpc",
        "cpm",
        "conversion_rate",
        
        # Temporal Data
        "start_time",
        "end_time",
        "create_time",
        "modify_time",
        
        # Status Information
        "status",
        "operation_status",
        "reject_reason",
        "review_result"
    ]

In [ ]:
# Filter functions
def get_filters():
    """Get all filters from user input"""
    print("\n=== 🎯 Set Your Filtering Criteria ===")
    filters = {}
    filter_name = []
    
    # 1. Date Range
    print("\n📅 Select Date Range:")
    print("1) Last 30 days")
    print("2) Last 90 days")
    print("3) Last 180 days")
    print("4) Custom range")
    date_choice = input("Enter your choice (1-4): ").strip()
    
    today = datetime.now()
    if date_choice == "1":
        start_date = (today - timedelta(days=30)).strftime('%Y-%m-%d')
        end_date = today.strftime('%Y-%m-%d')
    elif date_choice == "2":
        start_date = (today - timedelta(days=90)).strftime('%Y-%m-%d')
        end_date = today.strftime('%Y-%m-%d')
    elif date_choice == "3":
        start_date = (today - timedelta(days=180)).strftime('%Y-%m-%d')
        end_date = today.strftime('%Y-%m-%d')
    else:
        print("\nEnter custom date range:")
        start_date = input("Start date (YYYY-MM-DD): ").strip()
        end_date = input("End date (YYYY-MM-DD): ").strip()
    
    filters['start_date'] = start_date
    filters['end_date'] = end_date
    filter_name.append(f"dates_{start_date}_{end_date}")
    
    # 2. Ad Status
    print("\n📊 Select Ad Status:")
    print("1) All ads")
    print("2) Active ads only")
    print("3) Inactive ads only")
    status_choice = input("Enter your choice (1-3): ").strip()
    
    if status_choice == "2":
        filters['status'] = 'ACTIVE'
        filter_name.append('active')
    elif status_choice == "3":
        filters['status'] = 'INACTIVE'
        filter_name.append('inactive')
    
    # 3. Creative Type
    print("\n🎨 Select Creative Type:")
    print("1) All types")
    print("2) Single Image")
    print("3) Single Video")
    print("4) Carousel")
    creative_choice = input("Enter your choice (1-4): ").strip()
    
    creative_map = {
        "2": "SINGLE_IMAGE",
        "3": "SINGLE_VIDEO",
        "4": "CAROUSEL"
    }
    
    if creative_choice in creative_map:
        filters['creative_type'] = creative_map[creative_choice]
        filter_name.append(f"creative_{creative_map[creative_choice].lower()}")
    
    return filters, "_".join(filter_name)

In [ ]:
# Data extraction function
def extract_tiktok_ads_data(api_key=None, advertiser_id=None):
    """Extract TikTok ads data with filters"""
    print("=== TikTok Ads Data Extractor ===\n")
    
    # Setup API configuration if provided
    if api_key and advertiser_id:
        tiktok_config.setup(api_key, advertiser_id)
    
    # Check if API key is configured
    if not tiktok_config.API_KEY:
        api_key = input("Enter your TikTok API Key: ").strip()
        advertiser_id = input("Enter your Advertiser ID: ").strip()
        tiktok_config.setup(api_key, advertiser_id)
    
    # Get filters
    filters, filter_name = get_filters()
    
    try:
        print("\n⏳ Fetching data from TikTok Ads API...")
        
        # Build API request
        payload = {
            "advertiser_id": tiktok_config.ADVERTISER_ID,
            "start_date": filters['start_date'],
            "end_date": filters['end_date'],
            "fields": get_main_fields(),
            "page_size": 1000
        }
        
        if 'status' in filters:
            payload['filtering'] = [{"field": "status", "operator": "EQUALS", "value": filters['status']}]
        
        # Make API request
        response = requests.post(
            tiktok_config.BASE_URL,
            headers=tiktok_config.HEADERS,
            json=payload
        )
        
        if response.status_code == 200:
            data = response.json()
            if data['code'] == 0:  # Success
                df = pd.DataFrame(data['data']['list'])
                print(f"\n✅ Data fetched successfully! Found {len(df):,} ads")
                
                # Save to Excel
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                excel_path = os.path.join(output_dir, f"tiktok_ads_{filter_name}_{timestamp}.xlsx")
                
                with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
                    df.to_excel(writer, sheet_name='All Ads', index=False)
                    
                    # Add summary sheet
                    summary_data = {
                        'Metric': [
                            'Total Ads',
                            'Date Range',
                            'Total Impressions',
                            'Total Clicks',
                            'Total Spend',
                            'Average CTR',
                            'Report Generated'
                        ],
                        'Value': [
                            len(df),
                            f"{filters['start_date']} to {filters['end_date']}",
                            df['impressions'].sum(),
                            df['clicks'].sum(),
                            f"${df['spend'].sum():,.2f}",
                            f"{df['ctr'].mean():.2%}",
                            datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        ]
                    }
                    
                    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
                
                print(f"\n✅ Data saved to: {excel_path}")
                return df
            else:
                print(f"\n❌ API Error: {data['message']}")
        else:
            print(f"\n❌ HTTP Error: {response.status_code}")
            
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
    
    return None

In [ ]:
# Run the extraction
# Replace with your API key and advertiser ID
API_KEY = "your_api_key_here"
ADVERTISER_ID = "your_advertiser_id_here"

df = extract_tiktok_ads_data(API_KEY, ADVERTISER_ID)

## Data Analysis

After running the extraction, you can analyze your data here. Some example analyses:

In [ ]:
# Example: Performance Analysis
if df is not None:
    # Performance by creative type
    performance_by_type = df.groupby('creative_type').agg({
        'impressions': 'sum',
        'clicks': 'sum',
        'spend': 'sum',
        'ctr': 'mean'
    }).round(2)
    
    print("\nPerformance by Creative Type:")
    display(performance_by_type)
    
    # Top performing ads
    top_ads = df.nlargest(10, 'clicks')[[
        'ad_name', 'creative_type', 'impressions', 
        'clicks', 'spend', 'ctr'
    ]]
    
    print("\nTop 10 Performing Ads:")
    display(top_ads)